# CMS Open Payments Feature Engineering & Model Preparation

**Project:** AAI-540 Machine Learning Operations - Final Team Project  
**Dataset:** CMS Open Payments Program Year 2024 General Payments  
**Purpose:** Feature Engineering and Data Preparation for Anomaly Detection Models

---

## Table of Contents
1. [Environment Setup & Configuration](#setup)
2. [Data Loading from Datalake](#loading)
3. [Data Cleaning & Preprocessing](#cleaning)
4. [Data Partitioning (40/10/10/40)](#partitioning)
5. [Feature Engineering for ML Models](#ml-features)
6. [Outlier Detection](#outliers)
7. [Feature Store & Feature Group Setup](#feature-store)

---

## Project Objectives

- Prepare cleaned CMS Open Payments data for machine learning models
- Engineer features for Isolation Forest, XGBoost, and Autoencoder models
- Calculate risk scores and anomaly indicators for payment transactions
- Partition data according to project requirements (40% train, 10% test, 10% validation, 40% production)
- Store engineered features in Amazon SageMaker Feature Store

## 1. Environment Setup & Configuration

Setting up the environment with necessary libraries and AWS integration.

In [ ]:
# Install required packages
!pip install -r ../requirements.txt --quiet
!pip install boto3 sagemaker awswrangler pyathena --quiet

In [ ]:
# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from pathlib import Path
import sys
from scipy import stats

# AWS libraries
import boto3
import sagemaker
import awswrangler as wr
from pyathena import connect

from sagemaker.session import Session
from datetime import datetime
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import RobustScaler

# Add parent directory to path for custom modules
parent_dir = Path.cwd().parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Visualization settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("viridis")

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Libraries imported successfully")

In [ ]:
# Import custom utilities
try:
    from utils.visualizations import PaymentVisualizer
    visualizer = PaymentVisualizer()
    print("Visualization utilities imported successfully")
    use_visualizer = True
except ImportError as e:
    print(f"Could not import visualization utilities: {e}")
    print("Using standard plotting libraries only")
    use_visualizer = False

In [ ]:
# Restore AWS configuration from datalake setup notebook
%store -r bucket
%store -r region
%store -r database_name
%store -r table_name_parquet
%store -r s3_parquet_path
%store -r s3_athena_staging

# If variables not restored, set defaults matching datalake setup
try:
    # Test if variables exist
    test_vars = [bucket, region, database_name, table_name_parquet]
    
    print(f"AWS Configuration:")
    print(f"  Region: {region}")
    print(f"  S3 Bucket: {bucket}")
    print(f"  Database: {database_name}")
    print(f"  Table: {table_name_parquet}")
    print(f"  Parquet Path: {s3_parquet_path}")
    print(f"  Athena Staging: {s3_athena_staging}")
    
except NameError as e:
    print(f"Variables not found in store. Setting up from AWS configuration...")
    
    # Initialize AWS session
    boto_session = boto3.Session()
    region = boto_session.region_name
    
    # Get account information
    sts_client = boto3.client('sts')
    account_id = sts_client.get_caller_identity().get('Account')
    
    # Set configuration matching datalake setup
    #bucket = f"cmsopenpaymentsystems{account_id}"
    bucket = "cmsopenpaymentsystemslight" # changed to this to match Notebook 01 (JN)
    database_name = "cms_open_payments_light"
    table_name_parquet = "general_payments_parquet"
    
    # Define S3 paths
    cms_data_prefix = "cms-open-payments_light"
    parquet_data_prefix = f"{cms_data_prefix}/parquet"
    s3_parquet_path = f"s3://{bucket}/{parquet_data_prefix}"
    s3_athena_staging = f"s3://{bucket}/athena/staging"
    
    print(f"\nAWS Configuration (manual setup):")
    print(f"  Region: {region}")
    print(f"  Account ID: {account_id}")
    print(f"  S3 Bucket: {bucket}")
    print(f"  Database: {database_name}")
    print(f"  Table: {table_name_parquet}")
    print(f"  Parquet Path: {s3_parquet_path}")
    print(f"  Athena Staging: {s3_athena_staging}")
    print(f"\nNote: Please run the datalake setup notebook (01_setup_cms_datalake.ipynb) first for full setup.")

In [ ]:
# Validate prerequisites from notebook 01

print("PREREQUISITE CHECK: Validating variables from notebook 01")


required_from_nb01 = {
    'bucket': 'S3 bucket name',
    'region': 'AWS region',
    'database_name': 'Athena database name',
    'table_name_parquet': 'Parquet table name',
    's3_parquet_path': 'S3 parquet path',
    's3_athena_staging': 'Athena staging path'
}

missing_vars = []
for var_name, description in required_from_nb01.items():
    try:
        eval(var_name)
        print(f"{var_name}: {description}")
    except NameError:
        missing_vars.append((var_name, description))
        print(f"{var_name}: {description} - MISSING")

if missing_vars:
    
    print("  WARNING: Missing prerequisites from notebook 01")
    
    print(f"\nMissing {len(missing_vars)} required variables:")
    for var, desc in missing_vars:
        print(f"  - {var}: {desc}")
    print("\nThese will be set to defaults, but you should run notebook 01 first:")
    print("  01_setup_cms_datalake.ipynb")
    
else:
    print("\nAll prerequisites satisfied")
    

## 2. Data Loading from Datalake

In [ ]:
# Initialize Athena connection
athena_conn = connect(
    region_name=region,
    s3_staging_dir=s3_athena_staging
)

print("Athena connection established")

In [ ]:
# Option 1: Load full dataset from Parquet
load_full_dataset = True  # Set to True to load full dataset

if load_full_dataset:
    print("Loading full dataset from Parquet...")
    print("Note: This may take several minutes")
    
    df = wr.athena.read_sql_query(
        sql=f"SELECT * FROM {database_name}.{table_name_parquet}",
        database=database_name,
        ctas_approach=False
    )
    
    print(f"Full dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
else:
    print("Skipping full dataset load - will use sample queries instead")

In [ ]:
# Option 2: Load sample dataset for faster EDA
sample_size = 1_000_000 if load_full_dataset else 100_000

print(f"Loading sample dataset ({sample_size:,} rows)...")

sample_query = f"""
SELECT *
FROM {database_name}.{table_name_parquet}
LIMIT {sample_size}
"""

df = wr.athena.read_sql_query(
    sql=sample_query,
    database=database_name,
    ctas_approach=False
)

print(f"Sample dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")
print(f"Memory Usage: {df.memory_usage(deep=True).sum() / (1024**2):.2f} MB")

In [ ]:
# Preview the data
print("Dataset Preview:")
display(df.head(3))

print(f"\nDataset Info:")
print(f"  Shape: {df.shape}")
print(f"  Columns: {df.shape[1]}")
print(f"\nColumn Names:")
for i, col in enumerate(df.columns[:20], 1):
    print(f"  {i}. {col}")
if len(df.columns) > 20:
    print(f"  ... ({len(df.columns) - 20} more columns)")

## 3. Data Cleaning & Preprocessing

Prepare data for anomaly detection models by cleaning, standardizing, and selecting relevant features.

### 3.1 Initial Data Assessment

In [ ]:
initial_shape = df.shape
initial_columns = df.shape[1]

print(f"\nDataset Shape: {initial_shape}")
print(f"Total Records: {initial_shape[0]:,}")
print(f"Total Columns: {initial_columns}")

print(f"\nMemory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\nData Types:")
print(df.dtypes.value_counts())

print("\nMissing Values:")
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values(
    'Missing_Percent', ascending=False
)

print(f"Columns with missing values: {len(missing_summary)}/{len(df.columns)}")
if len(missing_summary) > 0:
    print("\nTop 10 columns with highest missingness:")
    display(missing_summary.head(10))

print(f"\nDuplicate Records: {df.duplicated().sum():,}")

### 3.2 Feature Selection for Anomaly Detection

In [ ]:
# Core features required for anomaly detection and risk scoring
CORE_PAYMENT_FEATURES = [
    'total_amount_of_payment_usdollars',
    'number_of_payments_included_in_total_amount',
    'date_of_payment'
]

CATEGORICAL_FEATURES = [
    'covered_recipient_type',
    'nature_of_payment_or_transfer_of_value',
    'form_of_payment_or_transfer_of_value',
    'physician_specialty',
    'recipient_state'
]

IDENTIFIER_FEATURES = [
    'covered_recipient_profile_id',
    'covered_recipient_npi',
    'applicable_manufacturer_or_applicable_gpo_making_payment_name'
]

RISK_INDICATOR_FEATURES = [
    'physician_ownership_indicator',
    'third_party_payment_recipient_indicator',
    'product_indicator'
]

# Verify feature availability
all_selected_features = (CORE_PAYMENT_FEATURES + CATEGORICAL_FEATURES + 
                         IDENTIFIER_FEATURES + RISK_INDICATOR_FEATURES)

available_features = [f for f in all_selected_features if f in df.columns]
missing_features = [f for f in all_selected_features if f not in df.columns]

print(f"\nFeature Availability:")
print(f"Selected: {len(all_selected_features)}")
print(f"Available: {len(available_features)}")
print(f"Missing: {len(missing_features)}")

if missing_features:
    print(f"\nMissing features: {missing_features}")

# Keep only selected features
df_selected = df[available_features].copy()

print(f"\nSelected dataset shape: {df_selected.shape}")
print(f"Columns retained: {df_selected.shape[1]}")
print(f"Records: {df_selected.shape[0]:,}")

### 3.3 Data Type Conversion and Validation

In [ ]:
# Convert payment amount to numeric
if 'total_amount_of_payment_usdollars' in df_selected.columns:
    df_selected['total_amount_of_payment_usdollars'] = pd.to_numeric(
        df_selected['total_amount_of_payment_usdollars'], errors='coerce'
    )
    # Handle negative amounts
    negative_count = (df_selected['total_amount_of_payment_usdollars'] < 0).sum()
    if negative_count > 0:
        print(f"Converting {negative_count:,} negative amounts to absolute values")
        df_selected['total_amount_of_payment_usdollars'] = df_selected['total_amount_of_payment_usdollars'].abs()

# Convert date to datetime
if 'date_of_payment' in df_selected.columns:
    df_selected['date_of_payment'] = pd.to_datetime(df_selected['date_of_payment'], errors='coerce')
    print(f"Converted date_of_payment to datetime")

# Convert payment count to integer
if 'number_of_payments_included_in_total_amount' in df_selected.columns:
    df_selected['number_of_payments_included_in_total_amount'] = pd.to_numeric(
        df_selected['number_of_payments_included_in_total_amount'], errors='coerce'
    ).fillna(1).astype('int64')

# Standardize indicator fields to binary
indicator_mapping = {'Yes': 1, 'Y': 1, 'No': 0, 'N': 0, 'Unknown': 0}

for col in RISK_INDICATOR_FEATURES:
    if col in df_selected.columns:
        if df_selected[col].dtype == 'object':
            df_selected[col] = df_selected[col].map(indicator_mapping).fillna(0).astype('int64')
            print(f"Standardized {col} to binary")

# Standardize state codes
if 'recipient_state' in df_selected.columns:
    df_selected['recipient_state'] = df_selected['recipient_state'].str.upper().str.strip()

print("\nData type conversion complete")
print("\nFinal data types:")
print(df_selected.dtypes)

### 3.4 Handle Missing Values

In [ ]:
missing_before = df_selected.isnull().sum().sum()

# Fill payment amount with median
if 'total_amount_of_payment_usdollars' in df_selected.columns:
    payment_median = df_selected['total_amount_of_payment_usdollars'].median()
    missing_amt = df_selected['total_amount_of_payment_usdollars'].isnull().sum()
    if missing_amt > 0:
        df_selected['total_amount_of_payment_usdollars'].fillna(payment_median, inplace=True)
        print(f"Filled {missing_amt:,} missing payment amounts with median: ${payment_median:.2f}")

# Fill date with forward/backward fill
if 'date_of_payment' in df_selected.columns:
    missing_dates = df_selected['date_of_payment'].isnull().sum()
    if missing_dates > 0:
        df_selected['date_of_payment'].fillna(method='ffill', inplace=True)
        df_selected['date_of_payment'].fillna(method='bfill', inplace=True)
        print(f"Filled {missing_dates:,} missing dates")

# Fill categorical fields with mode or Unknown
for col in df_selected.select_dtypes(include=['object']).columns:
    missing_count = df_selected[col].isnull().sum()
    if missing_count > 0:
        if col in ['covered_recipient_type', 'nature_of_payment_or_transfer_of_value']:
            mode_val = df_selected[col].mode()[0] if len(df_selected[col].mode()) > 0 else "Unknown"
            df_selected[col].fillna(mode_val, inplace=True)
        else:
            df_selected[col].fillna("Unknown", inplace=True)

# Fill numeric fields
for col in df_selected.select_dtypes(include=[np.number]).columns:
    missing_count = df_selected[col].isnull().sum()
    if missing_count > 0:
        if 'count' in col.lower() or 'number' in col.lower():
            df_selected[col].fillna(0, inplace=True)
        else:
            df_selected[col].fillna(df_selected[col].median(), inplace=True)

missing_after = df_selected.isnull().sum().sum()

print(f"\nMissing values before: {missing_before:,}")
print(f"Missing values after: {missing_after:,}")
print(f"Missing values resolved: {missing_before - missing_after:,}")

### 3.5 Remove Invalid Records

In [ ]:
records_before = len(df_selected)

# Remove duplicates
duplicates = df_selected.duplicated().sum()
if duplicates > 0:
    df_selected = df_selected.drop_duplicates()
    print(f"Removed {duplicates:,} duplicate records")

# Remove zero payment amounts
if 'total_amount_of_payment_usdollars' in df_selected.columns:
    zero_payments = (df_selected['total_amount_of_payment_usdollars'] == 0).sum()
    if zero_payments > 0:
        df_selected = df_selected[df_selected['total_amount_of_payment_usdollars'] > 0]
        print(f"Removed {zero_payments:,} zero payment records")

# Remove invalid dates
if 'date_of_payment' in df_selected.columns:
    invalid_dates = df_selected['date_of_payment'].isnull().sum()
    if invalid_dates > 0:
        df_selected = df_selected[df_selected['date_of_payment'].notnull()]
        print(f"Removed {invalid_dates:,} invalid date records")

# Remove records missing critical identifiers
if 'covered_recipient_profile_id' in df_selected.columns:
    missing_id = df_selected['covered_recipient_profile_id'].isnull().sum()
    if missing_id > 0:
        df_selected = df_selected[df_selected['covered_recipient_profile_id'].notnull()]
        print(f"Removed {missing_id:,} records with missing recipient ID")

# Reset index
df_selected = df_selected.reset_index(drop=True)

records_after = len(df_selected)
records_removed = records_before - records_after

print(f"\nRecords before: {records_before:,}")
print(f"Records after: {records_after:,}")
print(f"Records removed: {records_removed:,} ({records_removed/records_before*100:.2f}%)")
print(f"Data retention: {records_after/records_before*100:.2f}%")

# Update df with cleaned data
df = df_selected.copy()

### 3.6 Data Cleaning Summary

In [ ]:
# Summary statistics table
summary_data = {
    'Metric': [
        'Initial Records',
        'Final Records',
        'Records Removed',
        'Initial Columns',
        'Final Columns',
        'Columns Removed',
        'Missing Values',
        'Duplicates',
        'Data Completeness (%)'
    ],
    'Value': [
        f"{initial_shape[0]:,}",
        f"{df.shape[0]:,}",
        f"{initial_shape[0] - df.shape[0]:,}",
        initial_columns,
        df.shape[1],
        initial_columns - df.shape[1],
        f"{df.isnull().sum().sum():,}",
        f"{df.duplicated().sum():,}",
        f"{(1 - df.isnull().sum().sum()/(df.shape[0]*df.shape[1]))*100:.2f}"
    ]
}

summary_df = pd.DataFrame(summary_data)
display(summary_df)

# Feature categories table
feature_data = {
    'Category': ['Core Payment', 'Categorical', 'Risk Indicators', 'Identifiers'],
    'Count': [
        len([f for f in CORE_PAYMENT_FEATURES if f in df.columns]),
        len([f for f in CATEGORICAL_FEATURES if f in df.columns]),
        len([f for f in RISK_INDICATOR_FEATURES if f in df.columns]),
        len([f for f in IDENTIFIER_FEATURES if f in df.columns])
    ]
}

print("\nFeature Categories:")
display(pd.DataFrame(feature_data))

# Payment statistics table
if 'total_amount_of_payment_usdollars' in df.columns:
    payment_stats = df['total_amount_of_payment_usdollars']
    payment_data = {
        'Statistic': ['Total', 'Mean', 'Median', 'Min', 'Max'],
        'Amount ($)': [
            f"{payment_stats.sum():,.2f}",
            f"{payment_stats.mean():,.2f}",
            f"{payment_stats.median():,.2f}",
            f"{payment_stats.min():,.2f}",
            f"{payment_stats.max():,.2f}"
        ]
    }
    
    print("\nPayment Statistics:")
    display(pd.DataFrame(payment_data))

print("\nData is ready for feature engineering and model training")
display(df.head(3))

In [ ]:
# Create temporal features from date column
payment_date_col = 'date_of_payment'
if payment_date_col in df.columns:
    df['payment_year'] = df[payment_date_col].dt.year
    df['payment_month'] = df[payment_date_col].dt.month
    df['payment_quarter'] = df[payment_date_col].dt.quarter
    df['payment_dayofweek'] = df[payment_date_col].dt.dayofweek
    df['is_weekend'] = (df[payment_date_col].dt.dayofweek >= 5).astype('int64')
    print("Temporal features created successfully")
else:
    print(f"Warning: {payment_date_col} not found in dataframe")

print("Feature creation complete")

## 4. Data Partitioning

Split the data according to project requirements 40/10/10/40

In [ ]:
# data split (40/10/10/40)
print("Applying 40/10/10/40 data split...")

# fix random seed 
np.random.seed(42)

# create a random mask to assign each row to a category
perms = np.random.rand(len(df))

# map the random decimals to the four mandatory groups
df['dataset_usage'] = pd.cut(
    perms, 
    bins=[0, 0.4, 0.5, 0.6, 1.0], 
    labels=['train', 'test', 'validation', 'production']
)

# display the exact counts to confirm the split matches requirements
split_summary = df['dataset_usage'].value_counts().sort_index()
split_pct = df['dataset_usage'].value_counts(normalize=True).sort_index() * 100

print("\nFinal Partition Summary:")
for label in split_summary.index:
    print(f"  {label.capitalize()}: {split_summary[label]:,} rows ({split_pct[label]:.1f}%)")

## 5. Feature Engineering for ML Models

Create new features for predictive modeling.

### 5.1 Baseline Engineering
Data leak prevention. This isolates the training set to establish what "normal" behavior looks for each doctor/hospital before merging them back into the main dataset.

In [ ]:
# drop existing features if they exist to prevent error
cols_to_drop = ['hist_pay_count', 'hist_pay_total', 'hist_pay_avg', 
                'hist_pay_std', 'hist_pay_max', 'amt_to_avg_ratio', 'amt_to_max_ratio']

# only drop columns that are actually in the dataframe
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

# setup column names
target_id = 'covered_recipient_profile_id' 
payment_col = 'total_amount_of_payment_usdollars'
nature_col = 'nature_of_payment_or_transfer_of_value'
recipient_type_col = 'covered_recipient_type'
state_col = 'recipient_state'

# isolate training set to calculate historical "Normal" behavior
df_train = df[df['dataset_usage'] == 'train']
global_median = df_train[payment_col].median()

print(f"Calculating baselines for: {target_id}")

# calculate Recipient-Level historical metrics from training data
recipient_features = df_train.groupby(target_id).agg({
    payment_col: ['count', 'sum', 'mean', 'std', 'max']
}).reset_index()

# flatten columns so they are easy to reference
recipient_features.columns = [
    target_id, 'hist_pay_count', 'hist_pay_total', 'hist_pay_avg', 'hist_pay_std', 'hist_pay_max'
]

# merge features back to the main dataframe
# using left merge to ensure we don't lose the production/validation rows
df = df.merge(recipient_features, on=target_id, how='left')

# create ratio features 
df['amt_to_avg_ratio'] = df[payment_col] / df['hist_pay_avg']
df['amt_to_max_ratio'] = df[payment_col] / df['hist_pay_max']

# Impute NaNs (handling recipients not seen in training)
df['is_new_recipient'] = df['hist_pay_avg'].isnull().astype(int)
df['hist_pay_avg'] = df['hist_pay_avg'].fillna(global_median)
df['amt_to_avg_ratio'] = df['amt_to_avg_ratio'].fillna(df[payment_col] / global_median)

# final cleanup for remaining numerical NaNs
df[['hist_pay_count', 'hist_pay_total', 'hist_pay_std', 'hist_pay_max', 'amt_to_max_ratio']] = \
    df[['hist_pay_count', 'hist_pay_total', 'hist_pay_std', 'hist_pay_max', 'amt_to_max_ratio']].fillna(0)

### Statistical Deviation & Imputation

In [ ]:
# temporal features, extracting seasonal patterns
df['date_of_payment'] = pd.to_datetime(df['date_of_payment'])
df['payment_month'] = df['date_of_payment'].dt.month
df['is_weekend'] = (df['date_of_payment'].dt.dayofweek >= 5).astype(int)

# categorical risk indicators
risk_categories = ['Gift', 'Entertainment', 'Travel and Lodging']
df['is_high_risk_nature'] = df['nature_of_payment_or_transfer_of_value'].isin(risk_categories).astype(int)

print("Block 2 Complete: Contextual features created.")

# final verification, view the engineered data across the 40/10/10/40 split
cols_to_view = ['dataset_usage', payment_col, 'hist_pay_avg', 'amt_to_avg_ratio', 'is_new_recipient', 'is_weekend']
display(df.groupby('dataset_usage')[cols_to_view].head(3))

### Multi-Dimensional Anomaly Detection

In [ ]:
# handle missing raw payment amounts by filling with the global training median
# this ensures that rows with NaN payments don't break the outlier detection models
df[payment_col] = df[payment_col].fillna(global_median)

# recalculate the ratio for these newly filled rows
df['amt_to_avg_ratio'] = df['amt_to_avg_ratio'].fillna(df[payment_col] / global_median)

# check for any remaining NaNs across the entire feature set
final_nan_check = df[['hist_pay_avg', 'amt_to_avg_ratio', 'is_new_recipient', 'is_weekend']].isnull().sum()
print("Remaining NaN values per feature:")
print(final_nan_check)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Define the numerical features that need scaling
features_to_scale = ['total_amount_of_payment_usdollars', 'hist_pay_avg', 
                     'hist_pay_std', 'amt_to_avg_ratio']

scaler = StandardScaler()

# Fit only on training data to prevent leakage, then transform everything
df.loc[df['dataset_usage'] == 'train', [f"{c}_scaled" for c in features_to_scale]] = \
    scaler.fit_transform(df.loc[df['dataset_usage'] == 'train', features_to_scale])

# Apply the trained scaler to the rest of the data (val/test/prod)
non_train_mask = df['dataset_usage'] != 'train'
df.loc[non_train_mask, [f"{c}_scaled" for c in features_to_scale]] = \
    scaler.transform(df.loc[non_train_mask, features_to_scale])

## 6. Outlier Detection

Identify and analyze outlier payments using multiple detection methods.

In [ ]:
# Outlier detection using IQR method
if payment_col in df.columns:   
    
    amounts = df[payment_col].dropna()
    
    # Method 1: IQR (Interquartile Range)
    Q1 = amounts.quantile(0.25)
    Q3 = amounts.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_outliers = df[(df[payment_col] < lower_bound) | (df[payment_col] > upper_bound)]
    
    print(f"\nIQR Method:")
    print(f"  Q1 (25th percentile): ${Q1:,.2f}")
    print(f"  Q3 (75th percentile): ${Q3:,.2f}")
    print(f"  IQR: ${IQR:,.2f}")
    print(f"  Lower Bound: ${lower_bound:,.2f}")
    print(f"  Upper Bound: ${upper_bound:,.2f}")
    print(f"  Outliers Detected: {len(iqr_outliers):,} ({len(iqr_outliers)/len(df)*100:.2f}%)")
    
    # Method 2: Z-Score
    z_scores = np.abs(stats.zscore(amounts))
    z_threshold = 3
    z_outliers = df[np.abs(stats.zscore(df[payment_col].fillna(0))) > z_threshold]
    
    print(f"\nZ-Score Method (threshold={z_threshold}):")
    print(f"  Outliers Detected: {len(z_outliers):,} ({len(z_outliers)/len(df)*100:.2f}%)")
    
    # Method 3: Percentile-based
    percentile_99 = amounts.quantile(0.99)
    percentile_outliers = df[df[payment_col] > percentile_99]
    
    print(f"\nPercentile Method (99th percentile):")
    print(f"  Threshold: ${percentile_99:,.2f}")
    print(f"  Outliers Detected: {len(percentile_outliers):,} ({len(percentile_outliers)/len(df)*100:.2f}%)")

### Outlier Detection Methods

In [ ]:
# Analyze outlier characteristics
if payment_col in df.columns and len(iqr_outliers) > 0:
    print(f"\nPayment Amount Statistics for Outliers:")
    print(f"  Count: {len(iqr_outliers):,}")
    print(f"  Mean: ${iqr_outliers[payment_col].mean():,.2f}")
    print(f"  Median: ${iqr_outliers[payment_col].median():,.2f}")
    print(f"  Min: ${iqr_outliers[payment_col].min():,.2f}")
    print(f"  Max: ${iqr_outliers[payment_col].max():,.2f}")

    # Top outliers
    print(f"\nTop 10 Outliers by Payment Amount:")
    top_outliers = iqr_outliers.nlargest(10, payment_col)[
        [col for col in [payment_col, recipient_type_col, state_col, nature_col] 
         if col in iqr_outliers.columns]
    ]
    display(top_outliers)
    
    # Visualize outliers
    if use_visualizer and len(recipient_id_cols) > 0:
        # Create aggregated data for outlier visualization
        df_aggregated = df.groupby(recipient_id_cols[0]).agg({
            payment_col: ['count', 'sum', 'mean']
        }).reset_index()
        df_aggregated.columns = ['recipient_id', 'payment_count', 'total_amount', 'avg_amount']
        
        fig = visualizer.plot_outliers_analysis(df_aggregated, amounts, 
                                                lower_bound, upper_bound, 
                                                z_threshold=3)
        if fig:
            plt.show()

In [ ]:
# Interactive scatter plot - Payment amount vs Count
if payment_col and payment_col in df.columns and recipient_type_col and recipient_type_col in df.columns:
    if use_visualizer and len(recipient_id_cols) > 0:
        # Aggregate by recipient
        scatter_data = df.groupby([recipient_id_cols[0], recipient_type_col]).agg({
            payment_col: ['count', 'sum']
        }).reset_index()
        
        scatter_data.columns = ['recipient_id', 'recipient_type', 'payment_count', 'total_amount']
        
        # Use visualizer for interactive scatter
        fig = visualizer.create_interactive_scatter(
            scatter_data.head(1000),  # Limit for performance
            x_col='payment_count',
            y_col='total_amount',
            color_col='recipient_type',
            size_col='total_amount',
            hover_data=['recipient_id'],
            title='Payment Frequency vs Total Amount by Recipient Type'
        )
        fig.show()

In [ ]:
# Define MODEL_FEATURES for anomaly detection models
MODEL_FEATURES = [
    'total_amount_of_payment_usdollars',
    'hist_pay_avg',
    'amt_to_avg_ratio',
    'is_new_recipient',
    'payment_month',
    'is_weekend'
]

# Add optional risk indicators if they exist in the dataframe
if 'is_high_risk_nature' in df.columns:
    MODEL_FEATURES.append('is_high_risk_nature')
if 'physician_ownership_indicator' in df.columns:
    MODEL_FEATURES.append('physician_ownership_indicator')
if 'third_party_payment_recipient_indicator' in df.columns:
    MODEL_FEATURES.append('third_party_payment_recipient_indicator')

# Filter to only available features
model_features_available = [f for f in MODEL_FEATURES if f in df.columns]
print(f"\nModel features defined: {len(MODEL_FEATURES)}")
print(f"Model features available: {len(model_features_available)}")
AVAILABLE_FEATURES = available_features

# Store feature lists
%store AVAILABLE_FEATURES
%store MODEL_FEATURES
%store CORE_PAYMENT_FEATURES
%store CATEGORICAL_FEATURES
%store RISK_INDICATOR_FEATURES

# Store dataframe
%store df

print("\nVariables stored successfully:")
print(f"  AVAILABLE_FEATURES: {len(AVAILABLE_FEATURES)} features")
print(f"  MODEL_FEATURES: {len(MODEL_FEATURES)} features")
print(f"  df: {df.shape[0]:,} records x {df.shape[1]} columns")
print("\nThese variables can be retrieved in subsequent notebooks using %store -r")

### 6.1 Data Validation & Cleaning

Validate and clean data before Feature Store ingestion to prevent downstream issues.

In [ ]:
# 1. Check for and remove infinity values
print("\n1. Checking for infinity values...")
numeric_cols_df = df.select_dtypes(include=[np.number]).columns
inf_counts = {}
for col in numeric_cols_df:
    inf_count = np.isinf(df[col]).sum()
    if inf_count > 0:
        inf_counts[col] = inf_count
        
if inf_counts:
    print(f"Found infinity values in {len(inf_counts)} columns:")
    for col, count in list(inf_counts.items())[:5]:
        print(f"- {col}: {count:,} infinities")
    if len(inf_counts) > 5:
        print(f"... and {len(inf_counts) - 5} more columns")
    
    # Replace infinities with NaN for proper handling
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    print("Replaced all infinities with NaN")
else:
    print("No infinity values found")

# 2. Cap extreme outliers at 99.9th percentile
print("\n2. Capping extreme outliers at 99.9th percentile...")
capped_cols = []
for col in numeric_cols_df:
    if col in df.columns:
        q999 = df[col].quantile(0.999)
        q001 = df[col].quantile(0.001)
        
        # Check if any values exceed thresholds
        if (df[col] > q999).any() or (df[col] < q001).any():
            outlier_count = ((df[col] > q999) | (df[col] < q001)).sum()
            df[col] = df[col].clip(lower=q001, upper=q999)
            capped_cols.append((col, outlier_count))

if capped_cols:
    print(f"Capped outliers in {len(capped_cols)} columns:")
    for col, count in capped_cols[:5]:
        print(f"- {col}: {count:,} values capped")
    if len(capped_cols) > 5:
        print(f"... and {len(capped_cols) - 5} more columns")
else:
    print("No extreme outliers found")

# 3. Check feature availability
print("\n3. Validating engineered features...")
required_features = ['hist_pay_avg', 'amt_to_avg_ratio', 'is_new_recipient', 
                     'is_high_risk_nature', 'payment_month', 'is_weekend']
missing_features = [f for f in required_features if f not in df.columns]

if missing_features:
    print(f"WARNING: Missing {len(missing_features)} required features:")
    for f in missing_features:
        print(f"- {f}")
    print("These features should be created in section 8.1")
else:
    print(f"All {len(required_features)} required features present")

# 4. Final data quality summary
print("\n4. Final Data Quality Summary:")
print(f"   Total records: {len(df):,}")
print(f"   Total features: {len(df.columns)}")
print(f"   Missing value percentage: {(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100):.2f}%")
print(f"   Duplicate rows: {df.duplicated().sum():,}")

# Check numeric ranges
numeric_summary = df[numeric_cols_df].describe()
print(f"\nNumeric features range check:")
print(f"Min value across all numeric cols: {df[numeric_cols_df].min().min():.2f}")
print(f"Max value across all numeric cols: {df[numeric_cols_df].max().max():.2f}")
print("DATA VALIDATION COMPLETE - Ready for Feature Store ingestion")

## 7. Feature Store & Feature Group Setup

Store engineered features in Amazon SageMaker Feature Store for model training and inference.

### 7.1 Setup & Prepare Data

In [ ]:
import sagemaker
import time

from time import gmtime, strftime
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup

# Initialize Feature Store session
sagemaker_session = sagemaker.Session()
default_s3_bucket_name = sagemaker_session.default_bucket()
prefix = "cms-anomaly-detection-featurestore"
role_arn = sagemaker.get_execution_role()

# Create unique feature group names with timestamps
timestamp = strftime("%d-%H-%M-%S", gmtime())
cms_payments_feature_group_name = f"cms-payments-fg-{timestamp}"
payment_aggregates_feature_group_name = f"cms-payment-agg-fg-{timestamp}"

# Prepare data for ingestion
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")
        elif "datetime" in str(data_frame.dtypes[label]):
            data_frame[label] = data_frame[label].astype(str)
        elif "category" in str(data_frame.dtypes[label]):
            data_frame[label] = data_frame[label].astype(str)

# Define essential features - separate engineered from raw features
# REQUIRED engineered features (must exist)
required_engineered_features = [
    'hist_pay_count', 'hist_pay_total', 'hist_pay_avg', 'hist_pay_std', 'hist_pay_max',
    'amt_to_avg_ratio', 'amt_to_max_ratio', 'is_new_recipient',
    'payment_year', 'payment_month', 'payment_quarter', 'payment_dayofweek', 
    'is_weekend', 'is_high_risk_nature', 'dataset_usage'
]

# OPTIONAL raw features (use if available in dataset)
optional_raw_features = [
    'covered_recipient_profile_id', 'total_amount_of_payment_usdollars',
    'number_of_payments_included_in_total_amount', 'covered_recipient_type',
    'nature_of_payment_or_transfer_of_value', 'form_of_payment_or_transfer_of_value',
    'physician_specialty', 'recipient_state'
]

# Validate that REQUIRED engineered features exist
missing_engineered = [f for f in required_engineered_features if f not in df.columns]
if missing_engineered:
    print(f"\n  ERROR: Missing {len(missing_engineered)} required ENGINEERED features:")
    for f in missing_engineered:
        print(f"    - {f}")
    print("\nThese features should be created in section 8.1 (Feature Engineering)")
    print("Please run feature engineering cells before proceeding.")
    raise ValueError(f"Missing {len(missing_engineered)} required engineered features for Feature Store")

# Check which optional raw features are available
missing_raw = [f for f in optional_raw_features if f not in df.columns]
if missing_raw:
    print(f"\nNote: {len(missing_raw)} optional raw features not found in dataset (will skip):")
    for f in missing_raw:
        print(f"  - {f}")

# Combine all desired features
essential_features = required_engineered_features + optional_raw_features

# Filter to available columns only
selected_cols = [col for col in essential_features if col in df.columns]
print(f"\nFeature Selection Summary:")
print(f"  Original dataframe: {df.shape[1]} columns")
print(f"  Selected features: {len(selected_cols)} columns")
print(f"  Required engineered: {len([f for f in required_engineered_features if f in selected_cols])}/{len(required_engineered_features)}")
print(f"  Optional raw: {len([f for f in optional_raw_features if f in selected_cols])}/{len(optional_raw_features)}")
print(f"  Reduction: {((df.shape[1] - len(selected_cols)) / df.shape[1] * 100):.1f}% fewer columns")
print(f"  Benefit: Faster serialization, transfer, and storage")

df_payments = df[selected_cols].copy()
cast_object_to_string(df_payments)

current_time_sec = int(round(time.time()))
df_payments['EventTime'] = pd.Series([current_time_sec] * len(df_payments), dtype="float64")
record_identifier_feature_name = 'covered_recipient_profile_id'

print(f"\nData ready for ingestion:")
print(f"  Records: {len(df_payments):,}")
print(f"  Features: {len(df_payments.columns)} columns")
print(f"  Estimated time: ~{len(df_payments)/3000/60:.1f} minutes (at 3k rec/sec)")


### 7.2 Create Feature Groups

In [ ]:
# Create Feature Group instances
cms_payments_fg = FeatureGroup(
    name=cms_payments_feature_group_name, 
    sagemaker_session=sagemaker_session
)

payment_aggregates_fg = FeatureGroup(
    name=payment_aggregates_feature_group_name, 
    sagemaker_session=sagemaker_session
)

# Load feature definitions
cms_payments_fg.load_feature_definitions(data_frame=df_payments)

def wait_for_feature_group_creation(feature_group, max_wait=3600):
    status = feature_group.describe().get("FeatureGroupStatus")
    start_time = time.time()
    
    while status == "Creating":
        if time.time() - start_time > max_wait:
            raise TimeoutError("Feature Group creation timed out")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    
    if status != "Created":
        raise RuntimeError(f"Feature Group creation failed with status: {status}")

# Create Feature Group
cms_payments_fg.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}/payments",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role_arn,
    enable_online_store=True,
    description="CMS Open Payments data with engineered features"
)

wait_for_feature_group_creation(cms_payments_fg)
print(f"Feature Group created: {cms_payments_feature_group_name}")

### 7.3 Ingest & Verify

In [ ]:
import time
import numpy as np

start_time = time.time()

# Configuration for high-throughput ingestion
BATCH_SIZE = 100000  # Process in large batches
MAX_WORKERS = 50      # Increased from 10 for higher parallelism
MAX_PROCESSES = 8     # Utilize multiple CPU cores

# Split dataframe into batches
num_batches = int(np.ceil(len(df_payments) / BATCH_SIZE))
print(f"Ingesting {len(df_payments):,} records in {num_batches} batches")
print(f"Batch size: {BATCH_SIZE:,} | Workers: {MAX_WORKERS} | Processes: {MAX_PROCESSES}")

ingestion_results = []
for batch_idx in range(num_batches):
    batch_start = batch_idx * BATCH_SIZE
    batch_end = min((batch_idx + 1) * BATCH_SIZE, len(df_payments))
    df_batch = df_payments.iloc[batch_start:batch_end]
    
    batch_time = time.time()
    result = cms_payments_fg.ingest(
        data_frame=df_batch,
        max_workers=MAX_WORKERS,
        max_processes=MAX_PROCESSES,
        wait=True
    )
    batch_elapsed = time.time() - batch_time
    
    ingestion_results.append(result)
    records_processed = batch_end
    throughput = len(df_batch) / batch_elapsed
    print(f"Batch {batch_idx + 1}/{num_batches}: {len(df_batch):,} records in {batch_elapsed:.1f}s "
          f"({throughput:.0f} rec/sec) | Total: {records_processed:,}/{len(df_payments):,}")

total_time = time.time() - start_time
avg_throughput = len(df_payments) / total_time

print(f"Ingestion Complete!")
print(f"Total time: {total_time:.1f}s ({total_time/60:.2f} min)")
print(f"Records: {len(df_payments):,}")
print(f"Throughput: {avg_throughput:.0f} records/second")

In [ ]:
# Verify ingestion and store Feature Store configuration
boto_session = boto3.Session(region_name=region)

try:
    featurestore_runtime = boto_session.client(
        service_name="sagemaker-featurestore-runtime", 
        region_name=region
    )
    
    sample_id = str(int(df_payments[record_identifier_feature_name].iloc[0]))
    response = featurestore_runtime.get_record(
        FeatureGroupName=cms_payments_feature_group_name,
        RecordIdentifierValueAsString=sample_id
    )
    print(f"Verified online store access ({len(response.get('Record', []))} features retrieved)")
except Exception as e:
    print(f"  Online store not yet available: {str(e)[:40]}...")
    featurestore_runtime = None

# Get Feature Store metadata
fg_desc = cms_payments_fg.describe()
offline_store_config = fg_desc.get('OfflineStoreConfig', {})
s3_storage_config = offline_store_config.get('S3StorageConfig', {})
offline_s3_uri = s3_storage_config.get('ResolvedOutputS3Uri', 'N/A')

print(f"\nFeature Store Configuration:")
print(f"  Feature Group: {cms_payments_feature_group_name}")
print(f"  Offline Store URI: {offline_s3_uri}")
print(f"  Record Identifier: {record_identifier_feature_name}")

# Store configuration for downstream notebooks
%store cms_payments_feature_group_name
%store record_identifier_feature_name
%store featurestore_runtime
%store cms_payments_fg
%store region
%store bucket
%store s3_athena_staging
%store database_name
%store table_name_parquet
%store df

print("Variables stored for downstream notebooks:")

stored_vars = {
    'cms_payments_feature_group_name': cms_payments_feature_group_name,
    'record_identifier_feature_name': record_identifier_feature_name,
    'featurestore_runtime': 'Client object' if featurestore_runtime else None,
    'cms_payments_fg': 'FeatureGroup object',
    'region': region,
    'bucket': bucket,
    'database_name': database_name,
    'df': f'{df.shape[0]:,} rows x {df.shape[1]} cols'
}

for var, val in stored_vars.items():
    print(f"  {var}: {val}")

